## Tutorial 07 - Non linear Elliptic problem
**_Keywords: DEIM, POD-Galerkin_**

### 1. Introduction

In this tutorial, we consider a non linear elliptic problem in a two-dimensional spatial domain $\Omega=(0,1)^2$. We impose a homogeneous Dirichlet condition on the boundary $\partial\Omega$. The source term is characterized by the following expression
$$
g(\boldsymbol{x}; \boldsymbol{\mu}) = 100\sin(2\pi x_0)cos(2\pi x_1) \quad \forall \boldsymbol{x} = (x_0, x_1) \in \Omega.
$$

This problem is characterized by two parameters. The first parameter $\mu_0$ controls the strength of the sink term and the second parameter $\mu_1$ the strength of the nonlinearity. The range of the two parameters is the following:
$$
\mu_0,\mu_1\in[0.01,10.0]
$$
The parameter vector $\boldsymbol{\mu}$ is thus given by
$$
\boldsymbol{\mu} = (\mu_0,\mu_1)
$$
on the parameter domain
$$
\mathbb{P}=[0.01,10]^2.
$$


In order to obtain a faster approximation of the problem, we pursue a model reduction by means of a POD-Galerkin reduced order method. In order to preserve the affinity assumption the discrete empirical interpolation method will be used on the forcing term $g(\boldsymbol{x}; \boldsymbol{\mu})$.



### 2. Parametrized formulation

Let $u(\boldsymbol{\mu})$ be the solution in the domain $\Omega$.

The strong formulation of the parametrized problem is given by:
<center>for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})$ such that</center>

$$ -\nabla^2u(\boldsymbol{\mu})+\frac{\mu_0}{\mu_1}(\exp\{\mu_1u(\boldsymbol{\mu})\}-1)=g(\boldsymbol{x}; \boldsymbol{\mu})$$
<br>
    
The corresponding weak formulation reads:
<center>for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})\in\mathbb{V}$ such that</center>

$$a\left(u(\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)+c\left(u(\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)=f(v;\boldsymbol{\mu})\quad \forall v\in\mathbb{V}$$

where

* the function space $\mathbb{V}$ is defined as
$$
\mathbb{V} = \{v\in H_1(\Omega) : v|_{\partial\Omega}=0\}
$$
* the parametrized bilinear form $a(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$a(u, v;\boldsymbol{\mu})=\int_{\Omega} \nabla u\cdot \nabla v \ d\boldsymbol{x},$$
* the parametrized bilinear form $c(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$c(u, v;\boldsymbol{\mu})=\mu_0\int_{\Omega} \frac{1}{\mu_1}\big(\exp\{\mu_1u\} - 1\big)v \ d\boldsymbol{x},$$
* the parametrized linear form $f(\cdot; \boldsymbol{\mu}): \mathbb{V} \to \mathbb{R}$ is defined by
$$f(v; \boldsymbol{\mu})= \int_{\Omega}g(\boldsymbol{x}; \boldsymbol{\mu})v \ d\boldsymbol{x}.$$

The output of interest $s(\boldsymbol{\mu})$ is given by
$$s(\boldsymbol{\mu}) = \int_{\Omega} v \ d\boldsymbol{x}$$
is computed for each $\boldsymbol{\mu}$.

In [1]:
from dolfin import *
from rbnics import *

### 3. Affine Decomposition 

For this problem the affine decomposition is straightforward:
$$a(u,v;\boldsymbol{\mu})=\underbrace{1}_{\Theta^{a}_0(\boldsymbol{\mu})}\underbrace{\int_{\Omega}\nabla u \cdot \nabla v \ d\boldsymbol{x}}_{a_0(u,v)},$$
$$c(u,v;\boldsymbol{\mu})=\underbrace{\mu_0}_{\Theta^{c}_0(\boldsymbol{\mu})}\underbrace{\int_{\Omega}\frac{1}{\mu_1}\big(\exp\{\mu_1u\} - 1\big)v \ d\boldsymbol{x}}_{c_0(u,v)},$$
$$f(v; \boldsymbol{\mu}) = \underbrace{100}_{\Theta^{f}_0(\boldsymbol{\mu})} \underbrace{\int_{\Omega}\sin(2\pi x_0)cos(2\pi x_1)v \ d\boldsymbol{x}}_{f_0(v)}.$$
We will implement the numerical discretization of the problem in the class
```
class NonlinearElliptic(NonlinearEllipticProblem):
```
by specifying the coefficients $\Theta^{a}_*(\boldsymbol{\mu})$, $\Theta^{c}_*(\boldsymbol{\mu})$ and $\Theta^{f}_*(\boldsymbol{\mu})$ in the method
```
    def compute_theta(self, term):
```
and the bilinear forms $a_*(u, v)$, $c_*(u, v)$ and linear forms $f_*(v)$ in
```
    def assemble_operator(self, term):
```

In [2]:
@DEIM("online", basis_generation="Greedy")
@ExactParametrizedFunctions("offline")
class NonlinearElliptic(NonlinearEllipticProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        NonlinearEllipticProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.du = TrialFunction(V)
        self.u = self._solution
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)
        # Store the forcing term expression
        self.f = Expression("sin(2*pi*x[0])*sin(2*pi*x[1])", element=self.V.ufl_element())
        # Customize nonlinear solver parameters
        self._nonlinear_solver_parameters.update({
            "linear_solver": "mumps",
            "maximum_iterations": 20,
            "report": True
        })

    # Return custom problem name
    def name(self):
        return "NonlinearEllipticDEIM"

    # Return theta multiplicative terms of the affine expansion of the problem.
    @compute_theta_for_derivatives
    def compute_theta(self, term):
        mu = self.mu
        if term == "a":
            theta_a0 = 1.
            return (theta_a0,)
        elif term == "c":
            theta_c0 = mu[0]
            return (theta_c0,)
        elif term == "f":
            theta_f0 = 100.
            return (theta_f0,)
        elif term == "s":
            theta_s0 = 1.0
            return (theta_s0,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_derivatives
    def assemble_operator(self, term):
        v = self.v
        dx = self.dx
        if term == "a":
            du = self.du
            a0 = inner(grad(du), grad(v)) * dx
            return (a0,)
        elif term == "c":
            u = self.u
            mu = self.mu
            c0 = (exp(mu[1] * u) - 1) / mu[1] * v * dx
            return (c0,)
        elif term == "f":
            f = self.f
            f0 = f * v * dx
            return (f0,)
        elif term == "s":
            s0 = v * dx
            return (s0,)
        elif term == "dirichlet_bc":
            bc0 = [DirichletBC(self.V, Constant(0.0), self.boundaries, 1)]
            return (bc0,)
        elif term == "inner_product":
            du = self.du
            x0 = inner(grad(du), grad(v)) * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")


# Customize the resulting reduced problem
@CustomizeReducedProblemFor(NonlinearEllipticProblem)
def CustomizeReducedNonlinearElliptic(ReducedNonlinearElliptic_Base):
    class ReducedNonlinearElliptic(ReducedNonlinearElliptic_Base):
        def __init__(self, truth_problem, **kwargs):
            ReducedNonlinearElliptic_Base.__init__(self, truth_problem, **kwargs)
            self._nonlinear_solver_parameters.update({
                "report": True,
                "line_search": "wolfe"
            })

    return ReducedNonlinearElliptic

## 4. Main program

### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [3]:
mesh = Mesh("data/square.xml")
subdomains = MeshFunction("size_t", mesh, "data/square_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/square_facet_region.xml")

### 4.2. Create Finite Element space (Lagrange P1)

In [4]:
V = FunctionSpace(mesh, "Lagrange", 1)

### 4.3. Allocate an object of the NonlinearElliptic class

In [5]:
problem = NonlinearElliptic(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(0.01, 10.0), (0.01, 10.0)]
problem.set_mu_range(mu_range)

### 4.4. Prepare reduction with a POD-Galerkin method

In [6]:
reduction_method = PODGalerkin(problem)
reduction_method.set_Nmax(20, DEIM=21)
reduction_method.set_tolerance(1e-8, DEIM=1e-4)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


### 4.5. Perform the offline phase

In [7]:
reduction_method.initialize_training_set(50, DEIM=60)
reduced_problem = reduction_method.offline()

=                     DEIM preprocessing phase begins for                      =
=   { v_0 * (-1 + exp(f_9 * f_14)) / f_14 } * dx(<Mesh #0>[everywhere], {}),   =
=                                    where                                     =
=                                 f_14 = mu_1                                  =
=f_9 = solution of NonlinearEllipticDEIM (exact problem decorator: False, component: (None,))=
=               with id 49fc90be369b6ead57691b159f98bb7ecf8d1031               =

:::::::::::::::::::::::::::::::::::: DEIM 0 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (2.9415867058053458, 7.6843478690549265)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.359042e+00
  2 SNES Function norm 1.221701e+00
  3 SNES Function norm 1.097487e+00
  4 SNES Function norm 9.854588e-01
  5 SNES Function norm 7.492808e-01
  6 SNES Function norm 4.679661e-01
  7 SNES Function norm 9.243486e-02
  8 SNES Function norm 5.404048e-03
  9 SNES Fun

  3 SNES Function norm 2.727893e-02
  4 SNES Function norm 4.411125e-04
  5 SNES Function norm 1.193932e-07
  6 SNES Function norm 9.648930e-15
PETSc SNES solver converged in 6 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 15 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (3.139657513767789, 5.3829577685291)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 8.796057e-01
  2 SNES Function norm 3.961552e-01
  3 SNES Function norm 7.290726e-02
  4 SNES Function norm 3.289035e-03
  5 SNES Function norm 7.093787e-06
  6 SNES Function norm 3.321205e-11
PETSc SNES solver converged in 6 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 16 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (7.668083125315975, 4.600792920073519)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 7.847465e-01
  2 SNES Function norm 1.667786e-0

  2 SNES Function norm 1.172335e+00
  3 SNES Function norm 7.533126e-01
  4 SNES Function norm 2.404174e-01
  5 SNES Function norm 2.852449e-02
  6 SNES Function norm 4.911204e-04
  7 SNES Function norm 1.462920e-07
  8 SNES Function norm 1.353044e-14
PETSc SNES solver converged in 8 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 31 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (9.210917959140161, 3.5290287686898445)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 7.452983e-01
  2 SNES Function norm 1.730384e-01
  3 SNES Function norm 1.348275e-02
  4 SNES Function norm 9.013660e-05
  5 SNES Function norm 4.106171e-09
  6 SNES Function norm 4.175045e-15
PETSc SNES solver converged in 6 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 32 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (0.4717795689316696, 2.11788560863

  1 SNES Function norm 1.359289e+00
  2 SNES Function norm 1.222529e+00
  3 SNES Function norm 1.098941e+00
  4 SNES Function norm 9.873269e-01
  5 SNES Function norm 8.529939e-01
  6 SNES Function norm 5.439087e-01
  7 SNES Function norm 1.626816e-01
  8 SNES Function norm 1.719102e-02
  9 SNES Function norm 2.276981e-04
  10 SNES Function norm 3.955822e-08
  11 SNES Function norm 4.422967e-15
PETSc SNES solver converged in 11 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 48 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (4.25440233695105, 1.1579093926212656)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 4.750915e-02
  2 SNES Function norm 2.707187e-04
  3 SNES Function norm 1.168018e-08
  4 SNES Function norm 4.333081e-15
PETSc SNES solver converged in 4 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 49 ::::::::::::::::::::::::::::::::::::
e


find initial mu
initial maximum interpolation error = 0.12563364123789508
initial maximum interpolation relative error = 1.0

:::::::::::::::::::::::::::::::::: DEIM N = 0 ::::::::::::::::::::::::::::::::::
solve interpolation for mu = (0.2969488703196016, 6.385801888346545)
compute and locate maximum interpolation error
update locations with (605,)


TypeError: initialize_global_indices(): incompatible function arguments. The following argument types are supported:
    1. (arg0: dolfin::Mesh, arg1: int) -> None

Invoked with: <dolfin.cpp.mesh.Mesh object at 0x7f3b78f5df10>, 1

### 4.6. Perform an online solve

In [ ]:
online_mu = (0.3, 9.0)
reduced_problem.set_mu(online_mu)
reduced_solution = reduced_problem.solve()
plot(reduced_solution, reduced_problem=reduced_problem)

### 4.7. Perform an error analysis

In [ ]:
reduction_method.initialize_testing_set(50, DEIM=60)
reduction_method.error_analysis()

### 4.8. Perform a speedup analysis

In [ ]:
reduction_method.speedup_analysis()